In [ ]:
import os
from os.path import isfile
import sys
import glob

os.environ["CUDA_VISIBLE_DEVICES"]="5"
#os.environ["TF_FORCE_GPU_ALLOW_GROWTH"]="true"
os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async"

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense, Concatenate, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import L1, L2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler

import mlflow
import mlflow.tensorflow

In [ ]:
mlflow.tensorflow.autolog()
mlflow.set_experiment("HSEL ae")

In [ ]:
files = glob.glob("/data/nature_run/fulldays_reduced_evenmore/*.npz")
print(files)

In [ ]:
#AE

a = ["/data/nature_run/fulldays_reduced_evenmore/20060615/hsel.npy",
"/data/nature_run/fulldays_reduced_evenmore/20061215/hsel.npy",
"/data/nature_run/fulldays_reduced_evenmore/20060515/hsel.npy",
"/data/nature_run/fulldays_reduced_evenmore/20060815/hsel.npy",
"/data/nature_run/fulldays_reduced_evenmore/20060915/hsel.npy",
"/data/nature_run/fulldays_reduced_evenmore/20060715/hsel.npy",
"/data/nature_run/fulldays_reduced_evenmore/20061015/hsel.npy",
"/data/nature_run/fulldays_reduced_evenmore/20060315/hsel.npy",
"/data/nature_run/fulldays_reduced_evenmore/20061115/hsel.npy"]

hsel_val = []

for i in a:
    # Keys: 'mh', 'hsel', 'scalar', 'table']
    tmp = np.load(i)
    hsel_val.append(tmp)
hsel = np.vstack(hsel_val)
print(hsel.shape)

In [ ]:
#AE

mins = np.min(hsel, axis=0)
maxs = np.max(hsel, axis=0)
hsel = (hsel - mins)/(maxs - mins)

In [ ]:
with tf.device('/cpu:0'):
    x_train = tf.convert_to_tensor(hsel, np.float32)
    y_train = tf.convert_to_tensor(hsel, np.float32)


In [ ]:
del hsel

In [ ]:

def build_autoencoder(input_dim, latent_dim):
    # Encoder
    input_layer = Input(shape=(input_dim,))
    act = 'relu'
    #act = 'gelu'
    #act = "sigmoid"
    
    encoder = Dense(72, activation=act)(input_layer)
    encoder = Dense(60, activation=act)(encoder)
    encoder = Dense(40, activation=act)(encoder)
    
    latent_space = Dense(latent_dim, activation="linear")(encoder)

    decoder = Dense(40, activation=act)(latent_space)
    decoder = Dense(60, activation=act)(decoder)
    decoder = Dense(72, activation=act)(decoder)
    output_layer = Dense(input_dim, activation='linear')(decoder)

    # Autoencoder model
    autoencoder = Model(inputs=input_layer, outputs=output_layer)
    encoder_model = Model(inputs=input_layer, outputs=latent_space)

    autoencoder.compile(optimizer='adam', loss='mse')
    
    
    return autoencoder, encoder_model
model, enc = build_autoencoder(1957, 32)
model.summary()

In [ ]:
with mlflow.start_run() as run:
    history = model.fit(x_train, y_train, epochs=1000, batch_size=2000, verbose=1)

In [ ]:
print()

In [ ]:
model.save("shitty_ae.keras")
enc.save("shitty_enc.keras")

In [ ]:
print()


In [ ]:
out = model.predict(x_train, batch_size=1000)

In [ ]:
ind = 0

plt.plot(x)

In [ ]:
def get_data(filelist):
    hsel_val = []
    scalar_val = []
    table_val = []

    for i in filelist:
        # Keys: 'mh', 'hsel', 'scalar', 'table']
        tmp = np.load(i)
        hsel_val.append(tmp["hsel"])
        scalar_val.append(tmp["scalar"])
        table_val.append(tmp["table"])

    return np.vstack(hsel_val), np.vstack(scalar_val), np.vstack(table_val)


train = ['/data/nature_run/fulldays_reduced_evenmore/all_20060815.npz',
         '/data/nature_run/fulldays_reduced_evenmore/all_20060915.npz',
         '/data/nature_run/fulldays_reduced_evenmore/all_20061015.npz',
         '/data/nature_run/fulldays_reduced_evenmore/all_20060515.npz',
         '/data/nature_run/fulldays_reduced_evenmore/all_20060715.npz',
         '/data/nature_run/fulldays_reduced_evenmore/all_20061115.npz',
         '/data/nature_run/fulldays_reduced_evenmore/all_20060315.npz', 
         '/data/nature_run/fulldays_reduced_evenmore/all_20061215.npz',
         '/data/nature_run/fulldays_reduced_evenmore/all_20060615.npz']

test = ['/data/nature_run/fulldays_reduced_evenmore/all_20060803.npz']

validation = ['/data/nature_run/fulldays_reduced_evenmore/all_20060803.npz']

hsel_train, scalar_train, table_train = get_data(train)
hsel_test, scalar_test, table_test = get_data(test)
hsel_val, scalar_val, table_val = get_data(validation)

In [ ]:
def get_data(filelist):
    hsel_val = []
    scalar_val = []
    table_val = []

    for i in filelist:
        # Keys: 'mh', 'hsel', 'scalar', 'table']
        tmp = np.load(i)
        hsel_val.append(tmp["hsel"])
        scalar_val.append(tmp["scalar"])
        table_val.append(tmp["table"])

    return np.vstack(hsel_val), np.vstack(scalar_val), np.vstack(table_val)


train = ['/data/nature_run/fulldays_reduced_evenmore/all_20060815.npz',
         '/data/nature_run/fulldays_reduced_evenmore/all_20060915.npz',
         '/data/nature_run/fulldays_reduced_evenmore/all_20061015.npz',
         '/data/nature_run/fulldays_reduced_evenmore/all_20060515.npz',
         '/data/nature_run/fulldays_reduced_evenmore/all_20060715.npz',
         '/data/nature_run/fulldays_reduced_evenmore/all_20061115.npz',
         '/data/nature_run/fulldays_reduced_evenmore/all_20060315.npz', 
         '/data/nature_run/fulldays_reduced_evenmore/all_20061215.npz',
         '/data/nature_run/fulldays_reduced_evenmore/all_20060615.npz']

test = ['/data/nature_run/fulldays_reduced_evenmore/all_20060803.npz']

validation = ['/data/nature_run/fulldays_reduced_evenmore/all_20060803.npz']

hsel_train, scalar_train, table_train = get_data(train)
hsel_test, scalar_test, table_test = get_data(test)
hsel_val, scalar_val, table_val = get_data(validation)

In [ ]:


# Presssure
spress_test = scalar_test[:, 3].reshape(-1, 1)
spress_train = scalar_train[:, 3].reshape(-1, 1)
spress_val = scalar_val[:, 3].reshape(-1, 1)

mins = np.min(spress_train, axis=0)
maxs = np.max(spress_train, axis=0)

np.savez("spress_scalar.npz", mins=mins, maxs=maxs)
spress_test = (spress_test - mins)/(maxs - mins)
spress_train = (spress_train - mins)/(maxs - mins)
spress_val = (spress_val - mins)/(maxs - mins)


# Labels
q_train = table_train[:, :, 1]
q_test = table_test[:, :, 1]
q_val = table_val[:, :, 1]

# Spectra

mins = np.min(hsel_train, axis=0)
maxs = np.max(hsel_train, axis=0)
np.savez("minimac_scaling_factors_hsel_v2.npz", mins=mins, maxs=maxs)

#s_factors = np.load("minimac_scaling_factors_hsel.npz")
#mins = s_factors['mins']
#maxs = s_factors['maxs']

hsel_train = np.nan_to_num(hsel_train)
hsel_test = np.nan_to_num(hsel_test)
hsel_val = np.nan_to_num(hsel_val)

hsel_train = (hsel_train - mins)/(maxs - mins)
hsel_test = (hsel_test - mins)/(maxs - mins)
hsel_val = (hsel_val - mins)/(maxs - mins)


with tf.device('/cpu:0'):
    # train
    x_train = {'rad': tf.convert_to_tensor(hsel_train, np.float32),
               'spress': tf.convert_to_tensor(spress_train, np.float32)}
    del hsel_train
    del spress_train
    
    y_train =  tf.convert_to_tensor(q_train, np.float32)
    del q_train

    
    # Val
    x_val = {'rad': tf.convert_to_tensor(hsel_val, np.float32),
             'spress': tf.convert_to_tensor(spress_val, np.float32)}
    del hsel_val
    del spress_val
    
    y_val =  tf.convert_to_tensor(q_val, np.float32)
    del q_val

    
    # Test
    x_test = {'rad': tf.convert_to_tensor(hsel_test, np.float32),
              'spress': tf.convert_to_tensor(spress_test, np.float32)}
    del hsel_test
    del spress_test
    
    y_test =  tf.convert_to_tensor(q_test, np.float32)
    del q_test


In [ ]:
def build_ae_model(config):
    # ATMS 22
    mh = Input(shape=(config["shape"],), name="rad")
    spress = Input(shape=(1,), name="spress")
    # [ha, hb, hc, hd, hw, mh]

    encoder = load_model("encoder_3_mae.keras")
    encoder.trainable = False
    #for layer in encoder.layers:
    #    layer.trainable = False

    
    enc = encoder(mh)
    x = Concatenate()([enc, spress])
    for i in range(config["num_layers"]):
        x =  Dropout(config["dropout"])(Dense(config["num_neurons"], 
                                             activation=config["activation"])(x))
    outputs = Dense(config['output'], name="Temp")(x)

    model = Model(inputs=[mh, spress], outputs=outputs, name="Temp")
    model.compile(optimizer="adam", loss='mae')

    return model

config = {'shape': 1957,
          'output': 72,
          'dropout': np.random.choice([0.01, 0.1, 0.2, 0.5]),
          'num_layers': np.random.choice([3, 4, 5, 10]),
          'num_neurons': np.random.choice([16, 32, 64, 128]),
          'activation': np.random.choice(['gelu', 'gelu', 'relu'])}

model = build_ae_model(config)
model.summary()


In [ ]:
def build_model(config):
    # ATMS 22
    mh = Input(shape=(config["shape"],), name="rad")
    spress = Input(shape=(1,), name="spress")
    # [ha, hb, hc, hd, hw, mh]
    
    x = Concatenate()([mh, spress])
    for i in range(config["num_layers"]):
        x =  Dropout(config["dropout"])(Dense(config["num_neurons"], 
                                             activation=config["activation"])(x))
    outputs = Dense(config['output'], name="Temp")(x)

    model = Model(inputs=[mh, spress], outputs=outputs, name="Temp")
    model.compile(optimizer="adam", loss='mae')

    return model

config = {'shape': 1957,
          'output': 72,
          'dropout': np.random.choice([0.01, 0.1, 0.2, 0.5]),
          'num_layers': np.random.choice([3, 4, 5, 10]),
          'num_neurons': np.random.choice([16, 32, 64, 128]),
          'activation': np.random.choice(['gelu', 'gelu', 'relu'])}

model = build_model(config)
model.summary()


In [ ]:
print("DING")

In [ ]:
# AE
num_runs = 100

for _ in range(num_runs):
    config = {'shape': 1957,
              'output': 72,
              'dropout': np.random.choice([0.01, 0.1, 0.2, 0.5]),
              'num_layers': np.random.choice([3, 4, 5, 10]),
              'num_neurons': np.random.choice([16, 32, 64, 128]),
              'activation': np.random.choice(['gelu', 'gelu', 'relu'])}
    
    ae = np.random.choice([False, True, True])
    if ae:
        model = build_ae_model(config)
    else:
        model = build_model(config)
    
    with mlflow.start_run() as run:
        model_checkpoint_callback = ModelCheckpoint(
            filepath=f"models/model_{run.info.run_id}.keras",
            save_weights_only=False,
            monitor='val_loss',
            mode='min',
            save_best_only=True)
        callback = EarlyStopping(monitor='val_loss', patience=10, verbose=0)
        mlflow.log_param("Autoencoder", ae)

        history = model.fit(x_train, y_train, epochs=1000, batch_size=2000,
                            validation_data=(x_val, y_val),
                            callbacks=[callback, model_checkpoint_callback])
        
        evaled_test = model.evaluate(x_train, y_train, batch_size=100)
        evaled_train = model.evaluate(x_test, y_test, batch_size=100)

        mlflow.log_metric("test_loss", evaled_test)
        mlflow.log_metric("train_loss", evaled_train)

        plt.plot(history.history["loss"])
        plt.plot(history.history["val_loss"])
        plt.title('Model Loss')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plotname = f"plots/loss_plot_{run.info.run_id}.png"
        plt.savefig(plotname)
        plt.close()

        # Log the plot as an artifact
        mlflow.log_artifact(plotname)
        #model_path = "modles"
        #mlflow.tensorflow.log_model(tf_saved_model_dir=model_path, artifact_path="model")
